In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import time
from datetime import datetime

# Load initial log data and preprocess
df = pd.read_csv('server_logs.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['log_length'] = df['log_message'].apply(len)

# Feature selection and scaling
features = ['log_length', 'status_code', 'response_time']
X = df[features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Model training with DBSCAN for anomaly detection
dbscan = DBSCAN(eps=0.5, min_samples=10)
df['cluster'] = dbscan.fit_predict(X_scaled)
df['anomaly'] = df['cluster'].apply(lambda x: 1 if x == -1 else 0)

# Monitor logs in real-time
def monitor_logs(new_log):
    new_log['log_length'] = len(new_log['log_message'])
    new_log_features = scaler.transform(new_log[features].values.reshape(1, -1))
    cluster = dbscan.fit_predict(new_log_features)
    is_anomaly = 1 if cluster == -1 else 0
    print(f"Log at {new_log['timestamp']}: {'Anomaly' if is_anomaly else 'Normal'}")
    return is_anomaly

# Simulate real-time log monitoring
for index, row in df.iterrows():
    time.sleep(1)
    is_anomaly = monitor_logs(row)
    if is_anomaly:
        print(f"ALERT: Anomalous log detected at {row['timestamp']}")

# Save the model for future real-time monitoring
import joblib
joblib.dump(dbscan, 'log_monitoring_dbscan_model.pkl')
print("Model saved as log_monitoring_dbscan_model.pkl")